Libraries

In [14]:
import pandas as pd
import requests
import pickle as pkl
import numpy as np
import pprint
import os

from dotenv import load_dotenv
from pathlib import Path

In [15]:
DATA_DIR = Path("..") / "data"

# Load the .env file
load_dotenv()

True

In [16]:
# Extracting data from Eurostat to choose the countries we are studying

# Satisfaction data
eurostat_satisfaction_query = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/ilc_pw01?lang=en&isced11=TOTAL&indic_wb=LIFESAT&sex=T&age=Y_GE16&geo=EA20&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&geo=IS&geo=NO&geo=CH&geo=UK&geo=ME&geo=MK&geo=AL&geo=RS&geo=TR&geo=XK&time=2022"
satisfaction_response = requests.get(eurostat_satisfaction_query)
satisfaction_json = satisfaction_response.json()

# GDP data
eurostat_gpd_query = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/tipsau10?lang=en&unit=CP_MNAC&geo=EA20&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&time=2022"
gpd_response = requests.get(eurostat_gpd_query)
gpd_json = gpd_response.json()

In [17]:
# Extracting the data from the json
countries_id = satisfaction_json["dimension"]["geo"]["category"]["index"]
satisfaction_values = satisfaction_json["value"]
countries_full_name = satisfaction_json["dimension"]["geo"]["category"]["label"]

gpd_values = gpd_json["value"]

In [18]:
# Creating a dictionary relating the country names to the country satisfaction values

satisfaction_values_dict = {}
satisfaction_per_country_code = {}
final_satisf_dict = {}

for id in countries_id.values():
    satisfaction_values_dict[id] = satisfaction_values.get(str(id))

for i, j in zip(countries_id, satisfaction_values_dict):
    satisfaction_per_country_code[i] = satisfaction_values_dict[j]

for i, j in zip(countries_full_name.values(), satisfaction_per_country_code):
    final_satisf_dict.update({i: satisfaction_per_country_code[j]})

In [19]:
# Creating a dictionary relating the country names to the country GDP values

gpd_values_dict = {}
gpd_per_country_code = {}
final_gpd_dict = {}

for id in countries_id.values():
    gpd_values_dict[id] = gpd_values.get(str(id))

for i, j in zip(countries_id, gpd_values_dict):
    gpd_per_country_code[i] = gpd_values_dict[j]

for i, j in zip(countries_full_name.values(), gpd_per_country_code):
    final_gpd_dict.update({i: gpd_per_country_code[j]})

In [20]:
# Creating a Dataframe with the info found
eurostat_data = pd.DataFrame(
    {
        "Country": final_satisf_dict.keys(),
        "Satisfaction": final_satisf_dict.values(),
        "GPD": final_gpd_dict.values(),
    }
)

eurostat_data = eurostat_data.drop(0)  # Here we delete the row of 23 countries
eurostat_data.loc[37, "Country"] = "Kosovo"

In [21]:
eurostat_data

,Country,Satisfaction,GPD
1,Belgium,7.6,554044.3
2,Bulgaria,5.6,167809.0
3,Czechia,7.4,6786742.0
4,Denmark,7.5,2831643.9
5,Germany,6.5,3876810.0
6,Estonia,7.2,36011.1
7,Ireland,7.4,506282.4
8,Greece,6.7,206620.4
9,Spain,7.1,1346377.0
10,France,7.0,2639092.0


In [22]:
# Transforming info to make it easy to work with
# Converting all coins to Euro to be able to study them
frankfurter_url = "https://api.frankfurter.app"
ffdate = "2022-12-21"
non_euro_coins = ["RON", "HUF", "CHF", "CZK", "SEK", "PLN", "DKK", "BGN"]
conversors = {}

hrk = (
    7,
    53450,
)  # since 2023 Croatia has been using the euro, so we take the last exchange rate from 2022

hrk = float(hrk[0])

In [23]:
for coin in non_euro_coins:
    endpoint = f"{frankfurter_url}/{ffdate}?to={coin}"
    conversor_dict = requests.get(endpoint).json()
    conversor = conversor_dict["rates"][coin]
    conversors[coin] = conversor

conversors["HRK"] = hrk

non_euro_countries_coins = {
    "RON": "Romania",
    "HUF": "Hungary",
    "CHF": "Switzerland",
    "CZK": "Czechia",
    "SEK": "Sweden",
    "PLN": "Poland",
    "DKK": "Denmark",
    "BGN": "Bulgaria",
    "HRK": "Croatia",
}

In [24]:
# Filter the rows of the eurostat_data dataframe where Country is equal to non_euro_countries_coins

filtered_df = eurostat_data[
    eurostat_data["Country"].isin(non_euro_countries_coins.values())
]


# Iterating over the filtered rows and dividing the GPD value by the corresponding conversion in the conversors dictionary

# iterrows() returns the index and the row as a Series
for index, row in filtered_df.iterrows():

    country = row["Country"]

    """
    The following line of code is making a list of the keys of the non_euro_countries_coins dictionary to be able
    to make indexing. Then we are making a list from the values of the dictionary and we are getting the index of the
    country we are iterating.
    """
    coin = list(non_euro_countries_coins.keys())[
        list(non_euro_countries_coins.values()).index(country)
    ]

    gpd_value = row["GPD"]
    conversion_rate = conversors[coin]
    converted_gpd = gpd_value / conversion_rate
    eurostat_data.loc[index, "GPD"] = round(converted_gpd, 3)

In [25]:
eurostat_data

,Country,Satisfaction,GPD
1,Belgium,7.6,554044.300
2,Bulgaria,5.6,85800.695
3,Czechia,7.4,280235.445
4,Denmark,7.5,380699.637
5,Germany,6.5,3876810.000
6,Estonia,7.2,36011.100
7,Ireland,7.4,506282.400
8,Greece,6.7,206620.400
9,Spain,7.1,1346377.000
10,France,7.0,2639092.000


### <a id='toc2_2_2_'></a>[Standarizing and ponderating `satisfaction` and `GPD` data.](#toc0_)

For this study we are using a ponderated value based in this relation:

$$
Ponderatation = Satisfaction_{std} \cdot 0.6 + GPD_{std} \cdot 0.4 \tag{1}
$$

In [26]:
# Filling the NaN values with the mean of the column

eurostat_data["GPD"] = round(
    eurostat_data["GPD"].fillna(eurostat_data["GPD"].mean()), 3
)
eurostat_data["Satisfaction"] = eurostat_data["Satisfaction"].fillna(
    eurostat_data["Satisfaction"].mean()
)

For the standarization we are using the standarization formula:

$$
Z = \frac{x-\bar{x}}{std(x)} \tag{2}
$$

Once the `satisfaction` and `GPD` data are standarized with `(2)`, the `Ponderation` column is added using `(1)`.

In [27]:
# Standarizing the data
eurostat_data["Standarized Satisfaction"] = round(
    (eurostat_data["Satisfaction"] - eurostat_data["Satisfaction"].mean())
    / eurostat_data["Satisfaction"].std(),
    3,
)
eurostat_data["Standarized GPD"] = round(
    (eurostat_data["GPD"] - eurostat_data["GPD"].mean()) / eurostat_data["GPD"].std(),
    3,
)

# Ponderating the data
eurostat_data["Ponderation"] = (
    eurostat_data["Standarized Satisfaction"] * 0.6
    + eurostat_data["Standarized GPD"] * 0.4
)

In [38]:
eurostat_data.to_csv(DATA_DIR / "eurostat_data.csv", index=False)

In [28]:
eurostat_data.head()

,Country,Satisfaction,GPD,Standarized Satisfaction,Standarized GPD,Ponderation
1,Belgium,7.6,554044.300,0.873,-0.043,0.5066
2,Bulgaria,5.6,85800.695,-2.993,-0.651,-2.0562
3,Czechia,7.4,280235.445,0.486,-0.398,0.1324
4,Denmark,7.5,380699.637,0.680,-0.268,0.3008
5,Germany,6.5,3876810.000,-1.253,4.276,0.9586


In [29]:
airtable_base_url = "https://api.airtable.com/v0"
atTOKEN = os.getenv("AIRTABLE_API_KEY")

In [ ]:
# Upload of ponderations to Airtable

BASE_ID = "appZYI77p6tdWcbXo"
TABLE_ID1 = "tblfPPHv9mkLBVYK2"

headers = {"Authorization": f"Bearer {atTOKEN}", "Content-Type": "application/json"}

ponderation_table_endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID1}"

for i in range(0, eurostat_data.shape[0], 10):
    try:

        datos_subir = {
            "records": [
                {"fields": eurostat_data.iloc[j, :].to_dict()} for j in range(i, i + 10)
            ],
            "typecast": True,
        }

    except:

        datos_subir = {
            "records": [
                {"fields": eurostat_data.iloc[j, :].to_dict()}
                for j in range(i, eurostat_data.shape[0])
            ],
            "typecast": True,
        }
    response = requests.post(
        url=ponderation_table_endpoint, json=datos_subir, headers=headers
    )

In [30]:
# Choosing our countries to study
top_3_countries = (
    eurostat_data.sort_values("Ponderation", ascending=False)
    .head(3)
    .reset_index()
    .drop("index", axis=1)
)

top_3_countries

,Country,Satisfaction,GPD,Standarized Satisfaction,Standarized GPD,Ponderation
0,Switzerland,8.0,586778.948,1.646,0.000,0.9876
1,Germany,6.5,3876810.000,-1.253,4.276,0.9586
2,France,7.0,2639092.000,-0.287,2.667,0.8946


## <a id='toc3_2_'></a>[Get the target cities](#toc0_)

From the result of the previous cell, we choose the capitals of the countries with the highest ponderation to study them.
To obtain the capitals and their coordinates, we use the `countryinfo` and `geopy` libraries.

We will make the functions `get_capital()` and `get_lat_long` for building a dictionary that contains the info of the capitals and their coordinates, and a list of the capitals for the web-scrapping part.

In [31]:
from countryinfo import CountryInfo
from geopy.geocoders import Nominatim


def get_capital(country):
    info_pais = CountryInfo(country)
    return info_pais.capital()


def get_lat_long(country):
    geolocator = Nominatim(
        user_agent="my_app"
    )  # Replace "my_app" with your desired user agent
    info_pais = CountryInfo(country)
    location = geolocator.geocode(info_pais.capital())
    return location.latitude, location.longitude


final_cities = dict()
cities = list()

for i in range(3):
    capital = get_capital(top_3_countries.loc[i, "Country"])
    lat, long = get_lat_long(top_3_countries.loc[i, "Country"])

    final_cities[capital] = {"lat": lat, "lon": long}
    cities.append(capital)

In [32]:
pprint.pprint(final_cities)
print()
print(cities)

{'Berlin': {'lat': 52.5170365, 'lon': 13.3888599},
 'Bern': {'lat': 46.9484742, 'lon': 7.4521749},
 'Paris': {'lat': 48.8534951, 'lon': 2.3483915}}

['Bern', 'Berlin', 'Paris']


In [33]:
with open(DATA_DIR / "final_cities.pkl", "bw") as f:
    pkl.dump(final_cities, f)

with open(DATA_DIR / "cities.pkl", "bw") as f:
    pkl.dump(cities, f)

# <a id='toc4_'></a>[Weather information](#toc0_)

In [34]:
# Data extraction of current weather in each country

# Get the API key from the environment variable
API_KEY_opw = os.getenv("OPENWEATHERMAP_API_KEY")
search_url_opw = "https://api.openweathermap.org/data/2.5/weather"

weathers = []

for fcountry in final_cities.items():
    endpoint = f'{search_url_opw}?lat={fcountry[1]["lat"]}&lon={fcountry[1]["lon"]}&appid={API_KEY_opw}&units=metric'
    response = requests.get(url=endpoint)
    weather = response.json()
    weathers.append(weather)

In [35]:
# Creating a DataFrame with the info of weathers extracted
countries_id = []
cities = []
mains = []
temperatures = []
temps_min = []
temps_max = []
pressures = []
humidities = []
wind_speeds = []

for weather in weathers:

    countries_id.append(weather["sys"].get("country"))
    cities.append(weather.get("name"))
    mains.append(weather["weather"][0].get("main"))
    temperatures.append(weather["main"].get("temp"))
    temps_min.append(weather["main"].get("temp_min"))
    temps_max.append(weather["main"].get("temp_max"))
    pressures.append(weather["main"].get("pressure"))
    humidities.append(weather["main"].get("humidity"))
    wind_speeds.append(weather["wind"].get("speed"))

In [37]:
# Creating the DataFrame

weather_data = pd.DataFrame(
    {
        "Country": countries_id,
        "City": cities,
        "Main": mains,
        "Temperature (ºC)": temperatures,
        "Minimun temperature (ºC)": temps_min,
        "Maximun temperature (ºC)": temps_max,
        "Pressure (hPa)": pressures,
        "Humidity (%)": humidities,
        "Wind speed (meter/sec)": wind_speeds,
    }
)

weather_data.to_csv(DATA_DIR / "weather_data.csv", index=False)

In [39]:
weather_data

,Country,City,Main,Temperature (ºC),Minimun temperature (ºC),Maximun temperature (ºC),Pressure (hPa),Humidity (%),Wind speed (meter/sec)
0,CH,Bern,Clouds,5.26,2.41,6.42,1016,68,6.71
1,DE,Alt-Kölln,Clouds,7.12,5.02,8.37,1002,76,7.60
2,FR,Paris,Clouds,7.17,6.34,8.77,1014,82,5.14


In [ ]:
# Upload of current weather data to Airtable
TABLE_ID2 = "tblaqVEaHP4edqx7T"

headers = {"Authorization": f"Bearer {atTOKEN}", "Content-Type": "application/json"}

weather_table_endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID2}"

for i in range(0, weather_data.shape[0], 10):
    try:

        datos_subir = {
            "records": [
                {"fields": weather_data.iloc[j, :].to_dict()} for j in range(i, i + 10)
            ],
            "typecast": True,
        }

    except:

        datos_subir = {
            "records": [
                {"fields": weather_data.iloc[j, :].to_dict()}
                for j in range(i, weather_data.shape[0])
            ],
            "typecast": True,
        }
    response = requests.post(
        url=weather_table_endpoint, json=datos_subir, headers=headers
    )

# <a id='toc7_'></a>[Get a interest places dataset](#toc0_)

In [40]:
# Data extraction of interesting places (museums, colleges, churchs)
categories_data = pd.read_csv(DATA_DIR / "categories id.csv")

categorie_Ids = categories_data[
    categories_data["Category Label"].isin(
        ["Museums", "Colleges and Universities", "Spiritual Centers"]
    )
]["Category ID"].to_list()

In [43]:
CLIENT_ID_fsq = "MMU5IDIAX0XGQO5W15DRXXHZSBWH0TRIYV3WT342FAC1EQBI"
CLIENT_SECRET_fsq = "3XN11U5ERZ0ZFFHXZPQLO0LKIPFRWFXAWEJFHKZZO12LAFJN"
API_KEY_fsq = os.getenv("FOURSQUARE_API_KEY")
search_url_fsq = "https://api.foursquare.com/v3/places/search"

In [44]:
fsq_ids = []
countries = []
localities = []
categories = []
names = []
locations = []
latitudes = []
longitudes = []
institutions = ["museum", "College and University", "Spiritual Center"]

for fcountry in final_cities.items():

    for institution in institutions:

        url_params = {
            "query": institution,
            "ll": f"{fcountry[1]['lat']},{fcountry[1]['lon']}",
            "open_now": None,
            "limit": 50,
            "radius": 10_000,
            "category": ",".join(categorie_Ids),
        }

        headers = {"accept": "application/json", "Authorization": API_KEY_fsq}
        response = requests.get(url=search_url_fsq, headers=headers, params=url_params)
        places_json = response.json()

        for value in range(len(places_json["results"])):

            fsq_ids.append(places_json["results"][value].get("fsq_id"))
            countries.append(places_json["results"][value]["location"].get("country"))
            localities.append(places_json["results"][value]["location"].get("locality"))
            categories.append(
                places_json["results"][value]["categories"][0].get("name")
            )
            names.append(places_json["results"][value].get("name"))
            locations.append(
                places_json["results"][value]["location"].get("formatted_address")
            )
            latitudes.append(
                places_json["results"][value]["geocodes"]["main"].get("latitude")
            )
            longitudes.append(
                places_json["results"][value]["geocodes"]["main"].get("longitude")
            )

In [45]:
# Creating a fata frame with the infor extracted from Foursquare
places_data = pd.DataFrame(
    {
        "Fsq_Id": fsq_ids,
        "Country": countries,
        "Locality": localities,
        "Category": categories,
        "Name": names,
        "Location": locations,
        "Latitude": latitudes,
        "Longitude": longitudes,
    }
)

In [46]:
places_data.head()

,Fsq_Id,Country,Locality,Category,Name,Location,Latitude,Longitude
0,4be277911dd22d7f7f3094bd,CH,Bern,Museum,Bernisches Historisches Museum Einstein Museum,"Helvetiaplatz 5, 3005 Bern",46.942953,7.449349
1,4fd49753e4b03121ce7823b6,CH,Bern,Science Museum,Einsteinmuseum,"Helvetiaplatz 5, 3005 Bern",46.943129,7.449336
2,4ba3c1e5f964a520fe5b38e3,CH,Bern,Museum,Museum für Kommunikation,"Helvetiastrasse 16, 3005 Bern",46.941801,7.450036
3,4ca58ce1965c9c744881c7fa,CH,Bern,Museum,Naturhistorisches Museum,"Bernastrasse 15, 3005 Bern",46.942141,7.448943
4,4b4c5125f964a520dfb026e3,CH,Bern,Art Museum,Kunstmuseum Bern,"Hodlerstr. 8-12, 3011 Bern",46.951048,7.443254


In [47]:
# Cleaning data to eliminate the places that we are not looking for
places_data = places_data[
    ~places_data["Category"].isin(
        [
            "Non-Profit Organization",
            "Community and Government",
            "Management Consultant",
            "Library",
            "Bistro",
            "College Cafeteria",
            "Organization",
            "Structure",
            "Concert Hall",
            "Advertising Agency",
            "Hospital",
            "Spa",
            "Medical Center",
            "Yoga Studio",
        ]
    )
]
places_data = places_data.reset_index()
places_data = places_data.drop("index", axis=1)

In [48]:
places_data.to_csv(DATA_DIR / "places_data.csv", index=False)

In [ ]:
# Dataframe with the data about interesting places extracted
places_data

,Fsq_Id,Country,Locality,Category,Name,Location,Latitude,Longitude
0,4be277911dd22d7f7f3094bd,CH,Bern,Museum,Bernisches Historisches Museum Einstein Museum,"Helvetiaplatz 5, 3005 Bern",46.942953,7.449349
1,4fd49753e4b03121ce7823b6,CH,Bern,Science Museum,Einsteinmuseum,"Helvetiaplatz 5, 3005 Bern",46.943129,7.449336
2,4ba3c1e5f964a520fe5b38e3,CH,Bern,Museum,Museum für Kommunikation,"Helvetiastrasse 16, 3005 Bern",46.941801,7.450036
3,4ca58ce1965c9c744881c7fa,CH,Bern,Museum,Naturhistorisches Museum,"Bernastrasse 15, 3005 Bern",46.942141,7.448943
4,4b4c5125f964a520dfb026e3,CH,Bern,Art Museum,Kunstmuseum Bern,"Hodlerstr. 8-12, 3011 Bern",46.951048,7.443254
...,...,...,...,...,...,...,...,...
413,26ae9df35c1d43924dd11eb9,FR,None,Spiritual Center,Paroisse Saint Joseph,"Eglise Saint Joseph, 5 boulevard Charles de Ga...",48.936486,2.331618
414,4c1134176b7e2d7f7fe12835,FR,Paris,Spiritual Center,Temple Protestant de l'Oratoire du Louvre,"145 rue Saint-Honoré, 75001 Paris",48.861733,2.340070
415,4adcda09f964a5201a3421e3,FR,Paris,Church,American Church In Paris,"65 quai d'Orsay, 75007 Paris",48.862230,2.306729
416,4b530376f964a520958c27e3,FR,Paris,Church,Église Saint Nicolas du Chardonnet,"23 rue des Bernardins, 75005 Paris",48.849281,2.350241


In [ ]:
# Upload of interesting places to Airtable
TABLE_ID4 = "tblDuWR3nXGmk9OHl"

headers = {"Authorization": f"Bearer {atTOKEN}", "Content-Type": "application/json"}

places_table_endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID4}"

for i in range(0, places_data.shape[0], 10):
    try:

        datos_subir = {
            "records": [
                {"fields": places_data.iloc[j, :].to_dict()} for j in range(i, i + 10)
            ],
            "typecast": True,
        }

    except:

        datos_subir = {
            "records": [
                {"fields": places_data.iloc[j, :].to_dict()}
                for j in range(i, places_data.shape[0])
            ],
            "typecast": True,
        }
    response = requests.post(
        url=places_table_endpoint, json=datos_subir, headers=headers
    )